# METODOS NUMERICOS Y PROGRAMACION 4
## Tarea 2
##### Daniel Valencia - Computacion Cientifica - 2018-1
---

## Solucion numerica de EDPs

<ol>
    <li> Ecuaciones Parabolicas 
        <ul>
            <li> Metodo Explicito </li>
            <li> Metodo Implicitod </li>
            <li> Metodo Crank - Nicolson </li>
        </ul>
    </li>
</ol>

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import matplotlib.animation as animation
from matplotlib import rc
rc('animation', html='jshtml')

#### A partir de ahora se usaran las siguientes variables para los metodos
<ul>
    <li> c: Difusividad termica.</li>
    <li> L: Longitud de la barra.</li>
    <li> T: Tiempo final.</li>
    <li> h: Step-size que divide a la barra. </li>
    <li> k: Step-size que divide al tiempo.</li>
    <li> f: Condicion inicial U(x,0) = f(x).</li>
    <li> a: Condicion inicial de frontera en U(0,t).</li>
    <li> b: Condicion inicial de frontera en U(l,t).</li>
</ul>

In [2]:
def tridiag(a,b,c,N):
    A = np.zeros((N,N))
    
    np.fill_diagonal(A[:-1,1:],a)
    np.fill_diagonal(A,b)
    np.fill_diagonal(A[1:,:-1],c)
    
    return A

#### Error absoluto: 

In [3]:
def ErrorAb(a,b):
    n = len(a)
    c = []
    for i in range(n):
        c.append(np.absolute(a[i] - b[i]))
    return c

### - Implementacion en Python

In [4]:
def EDP_Imp_Matricial(c, L, T, h, k, f, a, b):
    
    r = c*k/h**2
    m = round(L/h) + 1
    n = round(T/k) + 1
    
    x = np.linspace(0, L, m)
    t = np.linspace(0,T,n)
    w = np.zeros((n,m))
    w[0] = f(x)
    w[:,0] = a(0) 
    w[:,-1] = b(L)
    
    W = tridiag(-r, 1+2*r, -r, m-2) 

    W_inv = np.linalg.inv(W)
    
    for i in range(n-1):
        B = w[i,1:-1].copy() 
        B[0] += r*w[i,0]
        B[-1] += r*w[i,-1]
        w[i+1,1:-1] = np.dot(W_inv, B)
        
    return w,x,t

### Ejercicios 1, 2, 3 y 4 del Burden (Pag 736)

### - Ejercicio 1:
Aproximar la solucion de la siguiente EDP usando el metodo implicito (Diferencias hacia atras): 

$$\frac{\partial U}{\partial t} (x,t)  - \frac{\partial ^2 U}{\partial x^2} (x,t)  = 0 \text{  ,  } 0 < x < 2 \text{  ,  } 0 \leq t$$
Con las condiciones frontera: 
$$U(0,t) = U(l,t) = 0 \text{  ,  } 0 < t$$
Y las condiciones iniciales: 
$$U(x,0) = sin\left(\frac{\pi}{2} x \right) \text{  ,  } 0 \leq x \leq 2$$

Usando $m=4$, $T = 0.1$ y $N = 2$. Compare sus resultados con la solucion analitica: $\displaystyle U(x,t) = e^{-(\pi^2 / 4)t} sin\left(\frac{\pi}{2} x \right)$

#### - Los datos para el ejemplo: 

In [5]:
Mat1, x1,t1 = EDP_Imp_Matricial(1, 2, 0.1, 0.5, 0.05, lambda x: np.sin(0.5*x*np.pi), lambda x: 0, lambda x:0)

#### - Solucion Exacta: 

In [6]:
def Exacta(x,t):
    n = len(x)
    sol = []
    for i in range(n):
        sol.append(((np.exp((-1)*(np.pi**2)*0.25*t))*np.sin((np.pi)*(0.5)*x[i])))
    return sol

In [7]:
Exa1 = Exacta(x1,0.05)
Err1 = ErrorAb(Mat1[-1],Exa1)
d1 = {'x': x1 , 'Aproximacion': Mat1[-1], 'Exacta': Exa1, 'Error': Err1}
df1 = pd.DataFrame(data=d1)
df1 = df1[['x','Aproximacion','Exacta','Error']]
df1

,x,Aproximacion,Exacta,Error
0,0.0,0.000000,0.000000e+00,0.000000e+00
1,0.5,0.566574,6.250375e-01,5.846386e-02
2,1.0,0.801256,8.839365e-01,8.268039e-02
3,1.5,0.566574,6.250375e-01,5.846386e-02
4,2.0,0.000000,1.082510e-16,1.082510e-16


#### Comparacion con el Burden (pag - 855)
El el Burden, a manera de solucion, se tiene la siguiente tabla: 
    <table>
  <tr>
    <th> Xi </th>
    <th> Wi </th> 
    <th> U(xi,tj)</th>
  </tr>
  <tr>
    <td>0.5</td>
    <td>0.566574</td> 
    <td>0.552493</td>
  </tr>
  <tr>
    <td>1.0</td>
    <td>0.801256</td> 
    <td>0.781344</td>
  </tr>
  <tr>
    <td>1.5</td>
    <td>0.566574</td> 
    <td>0.552493</td>
  </tr>
</table>

#### Como se puede apreciar, los resultados del algoritmo son iguales a las de las soluciones del libro.

### - Ejercicio 2:
Aproximar la solucion de la siguiente EDP usando el metodo implicito (Diferencias hacia atras): 

$$\frac{\partial U}{\partial t} (x,t)  - \frac{1}{16}\frac{\partial ^2 U}{\partial x^2} (x,t)  = 0 \text{  ,  } 0 < x < 2 \text{  ,  } 0 \leq t$$
Con las condiciones frontera: 
$$U(0,t) = U(l,t) = 0 \text{  ,  } 0 < t$$
Y las condiciones iniciales: 
$$U(x,0) = 2sin(2\pi x ) \text{  ,  } 0 \leq x \leq 2$$

Usando $m=3$, $T = 0.1$ y $N = 2$. Compare sus resultados con la solucion analitica: $ \displaystyle U(x,t) = 2e^{-(\pi^2 / 4)t} sin(2 x )$

#### Los datos para el ejemplo: 

In [8]:
Mat2, x2, t2 = EDP_Imp_Matricial(1/4, 2, 0.1, 1/1.5, 0.05, lambda x: 2*np.sin(2*x*np.pi), lambda x: 0, lambda x:0)

#### - Solucion exacta: 

In [9]:
def Exacta2(x,t):
    n = len(x)
    sol = []
    for i in range(n):
        sol.append(((2*np.exp((-1)*(np.pi**2)*0.25*t))*np.sin((np.pi)*2*x[i])))
    return sol

Exa2 = Exacta2(x2,0.05)

In [10]:
Exa2 = Exacta2(x2,0.05)
Err2 = ErrorAb(Mat2[-1],Exa2)

d2 = {'x': x2 , 'Aproximacion': Mat2[-1], 'Exacta': Exa2, 'Error': Err2}
df2 = pd.DataFrame(data=d2)
df2 = df2[['x','Aproximacion','Exacta','Error']]
df2

,x,Aproximacion,Exacta,Error
0,0.000000,0.000000,0.000000e+00,0.000000e+00
1,0.666667,-1.472996,-1.531023e+00,5.802670e-02
2,1.333333,1.472996,1.531023e+00,5.802670e-02
3,2.000000,0.000000,-8.660080e-16,8.660080e-16


#### - En el burden no se muestra la solucion para este problema, pero por el analisis del error absoluto se puede ver que es una muy buena aproximacion. 

### - Ejercicio 3: 
Repetir el ejercicio 1 usando Crank - Nicolson.

In [11]:
def EDP_CN_Matricial(c, L, T, h, k, f, a, b):
    
    r = c*k/h**2
    m = round(L/h) + 1
    n = round(T/k) + 1
    
    x = np.linspace(0, L, m)
    
    w = np.zeros((n,m))
    w[0] = f(x)
    w[:,0] = a(0) 
    w[:,-1] = b(L)
    
    A = tridiag(-r/2, 1+r, -r/2, m-2)
    B = tridiag(r/2, 1-r, r/2, m-2)

         
    A_inv = np.linalg.inv(A)


    for i in range(n-1):
        
        C = np.dot(B, w[i,1:-1])
        C[0]  += r/2*(w[i+1,0]  + w[i,0])
        C[-1] += r/2*(w[i+1,-1] + w[i,-1])

        w[i+1,1:-1] = np.dot(A_inv, C)
    
    return w, x

In [12]:
Mat3, x3 = EDP_CN_Matricial(1, 2, 0.1, 0.5, 0.05, lambda x: np.sin(0.5*x*np.pi), lambda x: 0, lambda x:0)

In [13]:
Exa1 = Exacta(x1,0.05)
Err3 = ErrorAb(Mat3[-1],Exa1)
d1 = {'x': x3 , 'Aproximacion': Mat3[-1], 'Exacta': Exa1, 'Error': Err3}
df1 = pd.DataFrame(data=d1)
df1 = df1[['x','Aproximacion','Exacta','Error']]
df1

,x,Aproximacion,Exacta,Error
0,0.0,0.000000,0.000000e+00,0.000000e+00
1,0.5,0.559251,6.250375e-01,6.578640e-02
2,1.0,0.790900,8.839365e-01,9.303602e-02
3,1.5,0.559251,6.250375e-01,6.578640e-02
4,2.0,0.000000,1.082510e-16,1.082510e-16


#### Comparacion con el Burden (pag - 855)
El el Burden, a manera de solucion, se tiene la siguiente tabla: 
    <table>
  <tr>
    <th> Xi </th>
    <th> Wi </th> 
    <th> U(xi,tj)</th>
  </tr>
  <tr>
    <td>0.5</td>
    <td>0.559251</td> 
    <td>0.552493</td>
  </tr>
  <tr>
    <td>1.0</td>
    <td>0.790901</td> 
    <td>0.781344</td>
  </tr>
  <tr>
    <td>1.5</td>
    <td>0.559252</td> 
    <td>0.552493</td>
  </tr>
</table>

#### Como se puede apreciar, los resultados del algoritmo son iguales a las de las soluciones del libro.

### - Ejercicio 4: 
Repetir el ejercicio 4 usando Crank - Nicolson.

In [14]:
Mat4, x4 = EDP_CN_Matricial(1/4, 2, 0.1, 1/1.5, 0.05, lambda x: 2*np.sin(2*x*np.pi), lambda x: 0, lambda x:0)

In [15]:
Exa2 = Exacta2(x4,0.05)
Err4 = ErrorAb(Mat4[-1],Exa2)
d4 = {'x': x2 , 'Aproximacion': Mat4[-1], 'Exacta': Exa2, 'Error': Err4}
df4 = pd.DataFrame(data=d4)
df4 = df4[['x','Aproximacion','Exacta','Error']]
df4

,x,Aproximacion,Exacta,Error
0,0.000000,0.000000,0.000000e+00,0.000000e+00
1,0.666667,-1.462951,-1.531023e+00,6.807149e-02
2,1.333333,1.462951,1.531023e+00,6.807149e-02
3,2.000000,0.000000,-8.660080e-16,8.660080e-16


#### - En el burden no se muestra la solucion para este problema, pero por el analisis del error absoluto se puede ver que es una muy buena aproximacion. 